In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import holodeck as holo
from holodeck.constants import MSOL

In [ ]:
# Load a parameter space instance to get default parameters (the input values here don't matter at all)
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, 10, 10, None)
# print(pspace.DEFAULTS)

In [ ]:
# Set our parameters that would vary across the parameter space / library
REDZ = 1.0
gsmf_phi0 = -2.5
gsmf_mchar0_log10 = 11.25
mmb_mamp_log10 = 9.0
mmb_scatter_dex = 0.4

gsmf = holo.sam.GSMF_Schechter(
    phi0=gsmf_phi0,
    phiz=pspace.DEFAULTS['gsmf_phiz'],
    mchar0_log10=gsmf_mchar0_log10,
    mcharz=pspace.DEFAULTS['gsmf_mcharz'],
    alpha0=pspace.DEFAULTS['gsmf_alpha0'],
    alphaz=pspace.DEFAULTS['gsmf_alphaz'],
)

mmbulge = holo.relations.MMBulge_KH2013(
    mamp_log10=mmb_mamp_log10,
    mplaw=pspace.DEFAULTS['mmb_plaw'],
    scatter_dex=mmb_scatter_dex,
)

# Construct a SAM just to get the mtot spacing, but this could also be constructed manually
_sam = holo.sam.Semi_Analytic_Model()
mtot = _sam.mtot


In [ ]:
# ---- Manually calculate the MBH mass function

# Convert from MBH masses to galaxy stellar masses
mstar = mmbulge.mstar_from_mbh(mtot, scatter=False)
# This is `dn_gal / dlog10(M_gal)`
ndens_gal = gsmf(mstar, REDZ)    # units of  [1/Mpc^3]

# Get the jacobian to convert differential elements  dM_gal / dM_bh
jac = mmbulge.dmstar_dmbh(mstar)   # [unitless]
# convert to dlog10(M_gal)/dlog10(M_star)
jac *= (mtot / mstar)
# convert density from stellar to MBH:   dn_bh / dlog10(M_bh)
ndens = ndens_gal * jac


# ---- Use the function included within the GSMF to calculate the MBH mass function

test_uniform = gsmf.mbh_mass_func(mtot, REDZ, mmbulge, scatter=False)
test_scatter = holo.utils.scatter_redistribute_densities(mtot, test_uniform, scatter=mmbulge._scatter_dex)


# ---- Plot

plt.loglog(mstar/MSOL, ndens_gal, label='GSMF')
plt.loglog(mtot/MSOL, ndens, alpha=0.75, label='Manual MBH mass func')
plt.loglog(mtot/MSOL, test_uniform, ls='--', lw=2.0, alpha=0.75, label='Built-in MBH mass func')
plt.loglog(mtot/MSOL, test_scatter, ls='-', lw=2.0, alpha=0.75, label='with scatter')

ax = plt.gca()
ax.set(ylim=[1e-8, 1e1])
ax.legend()
plt.show()